# Failed run

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns



from plotting.consts import (
    ATTRS_DICT, FADED_LINE_COLOR, LABEL_COLOR, LIGHT_GREY_TEXT,
    NULL_HEATMAP_COLOUR, TITLE_MAP, PLOT_WIDTH, PLOT_HEIGHT,
    FULL_PAGE_WIDTH
)

from plotting.utils import (
    # divergent_color_scale, get_shape_annotation,
    # get_text_annotation, get_arrow_annotation,
    grey_colorscale_discrete, grey_colorscale_discrete_N,
    invisible_colorbar, my_colorbar_subplot, standard_layout,
    grey_colorscale, my_colorbar, get_big_text_annotation
)

from param_scan.fns.config import config_rand
from param_scan.fns.post_process import PostProcess

from model.strategy_arrays import EqualResFreqBreakdownArray
from model.simulator import RunSingleTactic

from model.config_classes import SingleConfig

# Get data

In [3]:
RUN_TO_CHECK = 183
# RUN_TO_CHECK = 265

In [4]:
par_str = config_rand['par_str']
PP = PostProcess(par_str, '../src/param_scan/outputs')

# Runs where either ESFY better than ERFB or grid better than ERFB

ESFY better: 1
Grid better: 8

In [24]:
all_bad_runs = (
    PP.processed_df
    .loc[lambda df: 
         (df.c_E_maxContEL > df.c_R_maxContEL) |
         (df.max_grid_EL > df.c_R_maxContEL)
    ]
    .loc[:, [
             'c_E_maxContEL',
             'c_R_maxContEL',
             'max_grid_EL',
            ]
    ]
    .assign(
        ESFY_better = lambda df: df.c_E_maxContEL > df.c_R_maxContEL,
        grid_better = lambda df: df.max_grid_EL > df.c_R_maxContEL,
    )
    .sort_values('ESFY_better')
)

all_bad_runs

,c_E_maxContEL,c_R_maxContEL,max_grid_EL,ESFY_better,grid_better
run,,,,,
10,12,12,13,False,True
91,9,9,10,False,True
116,5,5,6,False,True
183,12,12,13,False,True
227,10,11,12,False,True
241,10,10,11,False,True
295,13,15,16,False,True
478,11,11,12,False,True
265,12,11,11,True,False


In [33]:
reruns = pd.DataFrame()

# for run_ind in [227, 478, 241, 91, 183]:
for run_ind in list(all_bad_runs.index):
    reruns = pd.concat(
        [
            reruns,
            # pd.read_csv(f'../outputs/re_run_conts/cont_101_300_{run_ind}.csv')
            pd.read_csv(f'../outputs/re_run_conts/cont_51_100_{run_ind}.csv')
        ])

(
    reruns
    .sort_values('worked', ascending=False)
    .reset_index(drop=True)
)

,c_R_maxContEL,max_grid_EL,run,worked
0,13,13,10,True
1,6,6,116,True
2,11,11,265,True
3,9,10,91,False
4,12,13,183,False
5,11,12,227,False
6,10,11,241,False
7,15,16,295,False
8,11,12,478,False


So runs 10 and 116 are actually ok.